In [ ]:
import time
import datetime
import pytz
import requests
import json
from base64 import b64encode
import os
from dotenv import load_dotenv
import sys

from google_cal_client import GoogleCalClient

In [134]:
class DataNotTrustedException(Exception):
    """Custom exception to indicate that the data returned should not be trusted."""
    pass

class TogglClient:
    """For reading toggl time entries via the toggl API."""

    def __init__(self):
        self.TOGGL_API_URL = 'https://api.track.toggl.com/api/v9/me'

        # Load environment variables from the .env file
        load_dotenv()

        # Get the API key from the environment variables
        api_token_string = os.getenv('API_KEY')

        # Check if the API key was loaded correctly
        if api_token_string is None:
            raise ValueError("API_KEY environment variable not set")

        api_token_string = api_token_string + ':api_token'

        self.headers = {
            'content-type': 'application/json',
            'Authorization' : 'Basic %s' %  b64encode(api_token_string.encode('utf-8')).decode("ascii")
          }

    def get_time_entries(self, verbose=False):
        unix_time_minus_two_hours = int(time.time()) - 14400
        
        response = requests.get(self.TOGGL_API_URL + f'/time_entries?since={unix_time_minus_two_hours}', headers=self.headers)

        # Ensure the request was successful
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
    
            # TODO: Add a debug mode that prints the details?
            # Or just delete this?
            # Pretty-print the JSON response
            #pretty_json = json.dumps(data, indent=4)
            #print(pretty_json)
        else:
            print(f"Request failed with status code {response.status_code}")

        # Look for non-null stop time to determine whether the time tracker entry actually 
        # has a stop time (if not, it's still in progress).

        if data and data[0].get('stop') is not None:
            entry_index = 0
        elif len(data) >= 2 and data[1].get('stop') is not None:
            entry_index = 1
        else:
            entry_index = -1
#            raise DataNotTrustedException("No valid entry found!")

        if entry_index in (0, 1):
            if verbose:
                print("Found toggl time entry:") 
                print(f"Description: {data[entry_index]['description']}")       
                print(f"Start: {data[entry_index]['start']}")       
                print(f"Stop: {data[entry_index]['stop']}")       

                return {"start": data[entry_index]['start'],
                        "stop" : data[entry_index]['stop'],
                        "description": data[entry_index]['description']}
        else:
            if verbose:
                print("No valid toggl time entry found.")
                return []



In [135]:
x = TogglClient()
x

In [ ]:
gcal = GoogleCalClient()
gcal

In [136]:
time_entry = x.get_time_entries(verbose=True)

No valid toggl time entry found.


In [44]:
if time_entry:
    print("Not Empty")
else:
    print("Empty")


Not Empty


In [130]:
gcal.create_event(time_entry)

Event created: https://www.google.com/calendar/event?eid=NnAxZWFwbmJidWtqM2ZjNTVnYmRzbWNhZGMgYW5kcmV3QGFuZHJld3dhbGRpcy5jb20


In [98]:
(91**2)%97

36

In [60]:
request_json = {'event_id': 11789171601940556, 'created_at': '2024-07-18T22:34:38.067Z', 'creator_id': 10638475, 'metadata': {'action': 'updated', 'event_user_id': '10638475', 'model': 'time_entry', 'model_owner_id': '10638475', 'project_id': '203719167', 'project_is_private': 'true', 'time_entry_id': '3532732305', 'workspace_id': '8239592'}, 'payload': {'at': '2024-07-18T22:34:37+00:00', 'billable': False, 'description': 'Google Cloud Function Development', 'duration': 218, 'duronly': True, 'id': 3532732305, 'permissions': ['update', 'remove'], 'pid': 203719167, 'project_id': 203719167, 'server_deleted_at': None, 'start': '2024-07-18T22:30:57Z', 'stop': '2024-07-18T22:34:35Z', 'tag_ids': [], 'tags': [], 'task_id': None, 'uid': 10638475, 'user_id': 10638475, 'wid': 8239592, 'workspace_id': 8239592}, 'subscription_id': 34375, 'timestamp': '2024-07-18T22:34:38.620514593Z', 'url_callback': 'https://us-central1-alert-aria-426417-f4.cloudfunctions.net/function-2'}

In [61]:
request_json

{'event_id': 11789171601940556,
 'created_at': '2024-07-18T22:34:38.067Z',
 'creator_id': 10638475,
 'metadata': {'action': 'updated',
  'event_user_id': '10638475',
  'model': 'time_entry',
  'model_owner_id': '10638475',
  'project_id': '203719167',
  'project_is_private': 'true',
  'time_entry_id': '3532732305',
  'workspace_id': '8239592'},
 'payload': {'at': '2024-07-18T22:34:37+00:00',
  'billable': False,
  'description': 'Google Cloud Function Development',
  'duration': 218,
  'duronly': True,
  'id': 3532732305,
  'permissions': ['update', 'remove'],
  'pid': 203719167,
  'project_id': 203719167,
  'server_deleted_at': None,
  'start': '2024-07-18T22:30:57Z',
  'stop': '2024-07-18T22:34:35Z',
  'tag_ids': [],
  'tags': [],
  'task_id': None,
  'uid': 10638475,
  'user_id': 10638475,
  'wid': 8239592,
  'workspace_id': 8239592},
 'subscription_id': 34375,
 'timestamp': '2024-07-18T22:34:38.620514593Z',
 'url_callback': 'https://us-central1-alert-aria-426417-f4.cloudfunctions.n

In [82]:
request_json['payload']['description']

'Google Cloud Function Development'

In [86]:
if 'payload' in request_json:
    if 'description' in request_json['payload']:
        description = request_json['payload']['description']
    else:
        description = '!! No description found !!'
        
    if 'start' in request_json['payload']:
        start = request_json['payload']['start']
    else:
        start = '!! No start time found !!'
print(f'Description: {description} - Start time: {start}')


Description: Google Cloud Function Development - Start time: 2024-07-18T22:30:57Z
